In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib as plt
from pathlib import Path

In [2]:
# Tuning Values

HPARAMS_DEFAULTS = 0
HPARAMS_PAPER = 1
HPARAMS_CUSTOM = 2
training_type = HPARAMS_PAPER


OP_RMSPROP = 0
OP_SGD = 1
op_type = OP_RMSPROP

if (training_type == HPARAMS_DEFAULTS):
    num_hidden_layers = 1
    num_hidden_layer_neurons = 32   # X
    dropout_rate = 0.4              # X
    batch_size = 32                 # X 
    optimizer_learning_rate = 0.01  # X  
    gradient_decay_rho = 0.9
    learning_rate_decay = 0.2       # X
    hidden_layer_activation_function = "relu"
    #-----
    op_type = OP_RMSPROP
    training_epochs = 30
    
elif (training_type == HPARAMS_PAPER):
    num_hidden_layers = 1
    num_hidden_layer_neurons = 256  # X
    dropout_rate = 0.4              # X
    batch_size = 2275               # X 
    optimizer_learning_rate = 3e-4  # X // 3e-4, or 3*(10^-4) = 0.0003
    gradient_decay_rho = 0.9
    learning_rate_decay = 0.2       # X
    hidden_layer_activation_function = "tanh"
    #-----
    op_type = OP_RMSPROP
    training_epochs = 30
    
elif (training_type == HPARAMS_CUSTOM):
    num_hidden_layers = 1
    num_hidden_layer_neurons = 135     # X
    dropout_rate = 0.2596709650832801  # X
    batch_size = 2275                  # X 
    optimizer_learning_rate = 0.0029044390072926946   # X // 3e-4, or 3*(10^-4) = 0.0003
    gradient_decay_rho = 0.7437864402038941
    learning_rate_decay = 0.39953909944182836         # X
    hidden_layer_activation_function = "tanh"
    #-----
    op_type = OP_RMSPROP
    training_epochs = 30

'''
Paper Results: 
Epoch 30/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7001 - loss: 0.5711 - val_accuracy: 0.9895 - val_loss: 0.1341

Custom Results:
Epoch 30/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8090 - loss: 0.4114 - val_accuracy: 0.9987 - val_loss: 0.0293


Trial 05 summary
Hyperparameters:
n_hidden: 1
n_neurons: 135
learning_rate: 0.0029044390072926946
dropout_rate: 0.2596709650832801
optimizer: rmsprop
rho: 0.7437864402038941
learning_rate_decay: 0.39953909944182836
decay_steps: 9028
Score: 1.0
'''

output_layer_activation_function = "sigmoid"
tf.keras.utils.set_random_seed(42)


use_gpu = True
if not use_gpu:
    import os
    #os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" #If the line below doesn't work, uncomment this line (make sure to comment the line below); it should help.
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
train_path = Path("../../feature_data/train/featureInput_noChampionRole_ZSCORED.csv")
test_path = Path("../../feature_data/test/featureInput_noChampionRole_ZSCORED.csv")

#names = ["Match Result", "Blue Top Player", "Blue Jungle Player", "Blue Middle Player", "Blue ADC Player", "Blue Support Player", "Red Top Player", "Red Jungle Player", "Red Middle Player", "Red ADC Player", "Red Support Player", "Blue Top Champion", "Blue Jungle Champion", "Blue Middle Champion", "Blue ADC Champion", "Blue Support Champion", "Red Top Champion", "Red Jungle Champion", "Red Middle Champion", "Red ADC Champion", "Red Support Champion", "Blue Player Cooperation", "Red Player Cooperation", "Blue vs Red Player", "Blue Champion Cooperation", "Red Champion Cooperation", "Blue vs Red Champion", "Blue Team Win Rate When Blue", "Red Team Win Rate When Red"]
names = ["bResult", "btPlayerRole", "bjPlayerRole", "bmPlayerRole", "baPlayerRole", "bsPlayerRole", "rtPlayerRole", "rjPlayerRole", "rmPlayerRole", "raPlayerRole", "rsPlayerRole", "btPlayerChampion", "bjPlayerChampion", "bmPlayerChampion", "baPlayerChampion", "bsPlayerChampion", "rtPlayerChampion", "rjPlayerChampion", "rmPlayerChampion", "raPlayerChampion", "rsPlayerChampion", "bCoopPlayer", "rCoopPlayer", "vsPlayer", "bCoopChampion", "rCoopChampion", "vsChampion", "bTeamColor", "rTeamColor"]

# Load our pre-processed training and test sets. 
# The skiprows value just removes the first row of the CSV which are labels instead of data.
train_data = (pd.read_csv(train_path, names=names, skiprows=1))
test_data  = (pd.read_csv( test_path, names=names, skiprows=1))

temp_data = (pd.read_csv(train_path, names=names, skiprows=1))


In [4]:
temp_data = temp_data.drop(['bResult'], axis=1)
temp_data.head()

,btPlayerRole,bjPlayerRole,bmPlayerRole,baPlayerRole,bsPlayerRole,rtPlayerRole,rjPlayerRole,rmPlayerRole,raPlayerRole,rsPlayerRole,...,raPlayerChampion,rsPlayerChampion,bCoopPlayer,rCoopPlayer,vsPlayer,bCoopChampion,rCoopChampion,vsChampion,bTeamColor,rTeamColor
0,0.252250,-0.113933,1.010727,0.434609,0.245249,0.160813,0.366454,0.119546,0.858685,0.036595,...,0.133663,-1.850269,0.366567,0.175634,0.127328,-0.187979,1.213948,-0.206860,1.103748,0.920580
1,0.107836,-0.258477,0.248977,-0.715370,-0.487638,0.034632,0.263361,-0.826679,0.262123,-1.356785,...,0.179959,0.263562,-0.418938,-0.767054,-1.588838,-1.739330,0.131607,-1.303218,-0.459587,-0.379011
2,-0.363338,0.955698,0.776230,-0.188098,1.248147,1.081633,0.469547,0.290217,0.775167,-2.793981,...,0.545983,-1.850269,-0.093366,-0.831687,1.020567,-0.054209,-0.383016,1.589459,0.383076,1.106054
3,0.178569,0.045067,-0.025138,-0.175134,0.548104,-0.455182,0.649960,-0.301999,3.340383,0.035027,...,2.043355,0.352566,0.180842,1.994147,-0.891270,0.085977,0.350546,0.363845,-0.198505,-0.502067
4,2.831061,0.661353,0.758049,1.205383,0.490318,-0.671590,-0.664476,-0.965114,-1.277006,-0.656151,...,-1.949637,-1.850269,0.571416,-0.580424,1.764771,0.582467,-1.015214,0.256906,0.451749,-0.436405


In [5]:
train_data.head()

,bResult,btPlayerRole,bjPlayerRole,bmPlayerRole,baPlayerRole,bsPlayerRole,rtPlayerRole,rjPlayerRole,rmPlayerRole,raPlayerRole,...,raPlayerChampion,rsPlayerChampion,bCoopPlayer,rCoopPlayer,vsPlayer,bCoopChampion,rCoopChampion,vsChampion,bTeamColor,rTeamColor
0,1,0.252250,-0.113933,1.010727,0.434609,0.245249,0.160813,0.366454,0.119546,0.858685,...,0.133663,-1.850269,0.366567,0.175634,0.127328,-0.187979,1.213948,-0.206860,1.103748,0.920580
1,0,0.107836,-0.258477,0.248977,-0.715370,-0.487638,0.034632,0.263361,-0.826679,0.262123,...,0.179959,0.263562,-0.418938,-0.767054,-1.588838,-1.739330,0.131607,-1.303218,-0.459587,-0.379011
2,1,-0.363338,0.955698,0.776230,-0.188098,1.248147,1.081633,0.469547,0.290217,0.775167,...,0.545983,-1.850269,-0.093366,-0.831687,1.020567,-0.054209,-0.383016,1.589459,0.383076,1.106054
3,0,0.178569,0.045067,-0.025138,-0.175134,0.548104,-0.455182,0.649960,-0.301999,3.340383,...,2.043355,0.352566,0.180842,1.994147,-0.891270,0.085977,0.350546,0.363845,-0.198505,-0.502067
4,1,2.831061,0.661353,0.758049,1.205383,0.490318,-0.671590,-0.664476,-0.965114,-1.277006,...,-1.949637,-1.850269,0.571416,-0.580424,1.764771,0.582467,-1.015214,0.256906,0.451749,-0.436405


In [6]:
len(train_data)

6823

In [7]:
len(test_data)

759

In [8]:
train_data.shape

(6823, 29)

In [9]:
test_data.shape

(759, 29)

In [10]:
train_data.dtypes

bResult               int64
btPlayerRole        float64
bjPlayerRole        float64
bmPlayerRole        float64
baPlayerRole        float64
bsPlayerRole        float64
rtPlayerRole        float64
rjPlayerRole        float64
rmPlayerRole        float64
raPlayerRole        float64
rsPlayerRole        float64
btPlayerChampion    float64
bjPlayerChampion    float64
bmPlayerChampion    float64
baPlayerChampion    float64
bsPlayerChampion    float64
rtPlayerChampion    float64
rjPlayerChampion    float64
rmPlayerChampion    float64
raPlayerChampion    float64
rsPlayerChampion    float64
bCoopPlayer         float64
rCoopPlayer         float64
vsPlayer            float64
bCoopChampion       float64
rCoopChampion       float64
vsChampion          float64
bTeamColor          float64
rTeamColor          float64
dtype: object

In [11]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[28]))
model.add(tf.keras.layers.Dropout(rate=dropout_rate))
for _ in range(num_hidden_layers):
    model.add(tf.keras.layers.Dense(num_hidden_layer_neurons, activation=hidden_layer_activation_function))
    model.add(tf.keras.layers.Dropout(rate=dropout_rate))
model.add(tf.keras.layers.Dense(1, activation=output_layer_activation_function))

if op_type == OP_RMSPROP:
    model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=optimizer_learning_rate, rho=gradient_decay_rho),
        metrics=["accuracy"])
else:
    model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.SGD(learning_rate=optimizer_learning_rate, decay=learning_rate_decay),
        metrics=["accuracy"])

In [12]:
y_train = train_data["bResult"]
y_test  =  test_data["bResult"]

In [26]:
y_train.head()

0    1
1    0
2    1
3    0
4    1
Name: bResult, dtype: int64

In [13]:
# Remove the match result column from the data.
X_train = train_data.drop("bResult", axis=1)
X_test  =  test_data.drop("bResult", axis=1)

In [25]:
X_train.head()

,btPlayerRole,bjPlayerRole,bmPlayerRole,baPlayerRole,bsPlayerRole,rtPlayerRole,rjPlayerRole,rmPlayerRole,raPlayerRole,rsPlayerRole,...,raPlayerChampion,rsPlayerChampion,bCoopPlayer,rCoopPlayer,vsPlayer,bCoopChampion,rCoopChampion,vsChampion,bTeamColor,rTeamColor
0,0.252250,-0.113933,1.010727,0.434609,0.245249,0.160813,0.366454,0.119546,0.858685,0.036595,...,0.133663,-1.850269,0.366567,0.175634,0.127328,-0.187979,1.213948,-0.206860,1.103748,0.920580
1,0.107836,-0.258477,0.248977,-0.715370,-0.487638,0.034632,0.263361,-0.826679,0.262123,-1.356785,...,0.179959,0.263562,-0.418938,-0.767054,-1.588838,-1.739330,0.131607,-1.303218,-0.459587,-0.379011
2,-0.363338,0.955698,0.776230,-0.188098,1.248147,1.081633,0.469547,0.290217,0.775167,-2.793981,...,0.545983,-1.850269,-0.093366,-0.831687,1.020567,-0.054209,-0.383016,1.589459,0.383076,1.106054
3,0.178569,0.045067,-0.025138,-0.175134,0.548104,-0.455182,0.649960,-0.301999,3.340383,0.035027,...,2.043355,0.352566,0.180842,1.994147,-0.891270,0.085977,0.350546,0.363845,-0.198505,-0.502067
4,2.831061,0.661353,0.758049,1.205383,0.490318,-0.671590,-0.664476,-0.965114,-1.277006,-0.656151,...,-1.949637,-1.850269,0.571416,-0.580424,1.764771,0.582467,-1.015214,0.256906,0.451749,-0.436405


In [14]:
import time
start_time = time.time()

In [15]:
# Train the model.

history = model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test),
    epochs=training_epochs,
    batch_size=batch_size)

Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step - accuracy: 0.5980 - loss: 0.6717 - val_accuracy: 0.9736 - val_loss: 0.3502
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7463 - loss: 0.5274 - val_accuracy: 0.9908 - val_loss: 0.2532
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7897 - loss: 0.4678 - val_accuracy: 0.9960 - val_loss: 0.2000
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8151 - loss: 0.4274 - val_accuracy: 1.0000 - val_loss: 0.1637
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.8260 - loss: 0.4029 - val_accuracy: 1.0000 - val_loss: 0.1373
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8514 - loss: 0.3696 - val_accuracy: 1.0000 - val_loss: 0.1185
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8582 - loss: 0.3570 - val_accuracy: 1.0000 - val_loss: 0.1025
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.8612 - loss: 0.3372 - val_accuracy: 1.0000 - val_loss: 0.0900

In [16]:
train_time = time.time() - start_time
train_time

5.117970705032349

In [17]:
import keras_tuner as kt

def build_model(hp):
    tune_n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=1)
    tune_n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    tune_learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
    tune_dropout_rate = hp.Float("dropout_rate", min_value=0.1, max_value=0.9)
    tune_optimizer = hp.Choice("optimizer", values=["sgd", "rmsprop"])
    
    tune_rho = hp.Float("rho", min_value=0.7, max_value=1.0)
    tune_learning_rate_decay = hp.Float("learning_rate_decay", min_value=0.01, max_value=0.99)
    tune_decay_steps = hp.Int("decay_steps", min_value=1000, max_value=100000)
    tune_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=tune_learning_rate,
        decay_steps=tune_decay_steps,
        decay_rate=tune_learning_rate_decay)
    
    if tune_optimizer == "sgd":
        tune_optimizer = tf.keras.optimizers.SGD(learning_rate=tune_lr_schedule)
    else:
        tune_optimizer = tf.keras.optimizers.RMSprop(learning_rate=tune_learning_rate, rho=tune_rho)
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=[28]))
    model.add(tf.keras.layers.Dropout(rate=tune_dropout_rate))
    for _ in range(tune_n_hidden):
        model.add(tf.keras.layers.Dense(tune_n_neurons, activation=hidden_layer_activation_function))
        model.add(tf.keras.layers.Dropout(rate=tune_dropout_rate))
    model.add(tf.keras.layers.Dense(1, activation=output_layer_activation_function))

    model.compile(
        loss="binary_crossentropy",
        optimizer=tune_optimizer,
        metrics=["accuracy"])

    return model

In [18]:
random_search_tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=10, overwrite=True, directory="results", project_name="tune_search", seed=42)
random_search_tuner.search(
    X_train, y_train, 
    validation_data=(X_test, y_test),
    epochs=training_epochs,
    batch_size=batch_size
)

Trial 10 Complete [00h 00m 08s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 01m 19s


In [19]:
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

C:\Users\scotty\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
C:\Users\scotty\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [20]:
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values

{'n_hidden': 5,
 'n_neurons': 25,
 'learning_rate': 0.0006562536901904111,
 'dropout_rate': 0.31750033150727264,
 'optimizer': 'rmsprop',
 'rho': 0.8055538767483648,
 'learning_rate_decay': 0.5471946718480031,
 'decay_steps': 7621}

In [21]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 00 summary
Hyperparameters:
n_hidden: 5
n_neurons: 25
learning_rate: 0.0006562536901904111
dropout_rate: 0.31750033150727264
optimizer: rmsprop
rho: 0.8055538767483648
learning_rate_decay: 0.5471946718480031
decay_steps: 7621
Score: 1.0


In [22]:
best_trial.metrics.get_last_value("val_accuracy")

np.float64(1.0)

In [23]:
#best_model.fit(X_train, y_train, epochs=10)
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 0.0306
